## Analysis
This project demonstrated the usage of pkdb for the test substance caffeine.
### Data Sources
- pkdb rest api : "http://0.0.0.0:8000/api/v1/"

### Outputs:
- data/processed/outputs.tsv
- data/processed/timecourses.tsv
- data/processed/interventions.tsv
- data/processed/individuals.tsv
- data/processed/groups.tsv

#### Groups and Individuals 
- data/processed/all_subjects.tsv

### Merged Dataframes
- data/processed/all_subjects.tsv

### Normalized Dataframes
- data/processed/all_subjects.tsv

### Changes
- 11-29-2018 : Started project

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import PkdbModel , Preprocessed
import pandas as pd


{'token': '4241a1580f26a51f470a716cbb84be132808f5a3'}


In [3]:
queries = ['outputs','timecourses','interventions','individuals','groups',"studies","substances"]
#queries = ["studies","substances"]
#queries = ["substances"]

### Query, Process and Save
Json data is loaded, transformed to dataframes, preprocessed and saved as tab seperated files.

In [4]:
for pk_instance in queries:
    pk_instance = PkdbModel(pk_instance)  
    pk_instance.load()
    pk_instance.preprocess()
    pk_instance.save()
    pk_instance.report()

____________________________________________________________
Name: outputs
Loaded: True
Preprocessed: True
saved: True
outputs were succsesfully saved to </home/janekg89/Develop/Pycharm_Projects/pkdb_analysis/data/1-preprocessed/outputs.tsv>
____________________________________________________________
Name: timecourses
Loaded: True
Preprocessed: True
saved: True
timecourses were succsesfully saved to </home/janekg89/Develop/Pycharm_Projects/pkdb_analysis/data/1-preprocessed/timecourses.tsv>
____________________________________________________________
Name: interventions
Loaded: True
Preprocessed: True
saved: True
interventions were succsesfully saved to </home/janekg89/Develop/Pycharm_Projects/pkdb_analysis/data/1-preprocessed/interventions.tsv>
____________________________________________________________
Name: individuals
Loaded: True
Preprocessed: True
saved: True
individuals were succsesfully saved to </home/janekg89/Develop/Pycharm_Projects/pkdb_analysis/data/1-preprocessed/individ

In [5]:
studies = PkdbModel('studies',destination='1-preprocessed')
studies.read()
frames = []
for number,row in studies.data.dropna(subset=["substances"]).iterrows():
    substances = [{"study":row["name"],"substance":x.strip()} for x in row["substances"].split(",")]
    frames.append(pd.DataFrame(substances))

study_substance_map = pd.concat(frames)
substance_study = study_substance_map.groupby(['substance'])['study'].agg(
    {
        'studies': lambda x: ','.join(x),
        'study_number': lambda x:len(x)
        })
substance_study = substance_study.reset_index()





/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  if sys.path[0] == '':


In [6]:
substance_study[substance_study["substance"]=="caf"]



,substance,studies,study_number


In [7]:
substances = PkdbModel('substances',destination='1-preprocessed')
substances.read()
substances.data

,name,intervention_number,timecourse_number,output_number,output_calculated_number,output_raw_number,interventions,outputs,outputs_calculated,timecourses
0,coffee,0,0,0,0,0,[],[],[],[]
1,tea,0,0,0,0,0,[],[],[],[]
2,kola nuts,0,0,0,0,0,[],[],[],[]
3,nicotine,0,0,0,0,0,[],[],[],[]
4,cocoa,0,0,0,0,0,[],[],[],[]
5,haemoglobin,0,0,0,0,0,[],[],[],[]
6,cholesterol,0,0,0,0,0,[],[],[],[]
7,fructosamine,0,0,0,0,0,[],[],[],[]
8,hba1c,0,0,0,0,0,[],[],[],[]
9,triglyceride,0,0,0,0,0,[],[],[],[]


In [8]:
substances = PkdbModel('substances',destination='1-preprocessed')
substances.read()
substances.data = substance_study.merge(substances.data, left_on="substance", right_on="name")



substances.data.sort_values(by="study_number", ascending=False, inplace=True)
substances.save()

In [9]:
this_data = PkdbModel('outputs',destination='1-preprocessed')
this_data.read()
this_data.data.groupby(["calculated"]).apply(len)

calculated
False    6940
True     4273
dtype: int64

In [10]:
this_data = PkdbModel('interventions',destination='1-preprocessed')
this_data.read()
this_data.data.groupby(["unit"]).apply(len)

unit
IU / hour / kilogram        1
gram                      320
gram / hour                 5
gram / kilogram            46
mole                        1
mole / hour / kilogram      4
mole / kilogram             1
yr                          1
dtype: int64

In [11]:
this_data = PkdbModel('outputs',destination='1-preprocessed')
this_data.read()
this_data.data.groupby(["measurement_type","unit"]).count()

allowed_users  \
measurement_type               unit                                                       
Glucose oral disposition index none                                                   6   
HOMA-IR                        none                                                   6   
Insulinogenic index            none                                                   6   
Matsuda index                  none                                                   6   
QUICKI                         none                                                   6   
age                            yr                                                     2   
amount                         gram                                                  96   
auc_end                        IU * hour / liter                                     11   
                               gram * hour / liter                                  881   
                               hour * mole / liter                                   14   
auc_inf                        IU * hour / liter                                     29   
                               gram * hour / liter                                  903   
                               hour * mole / liter                                   15   
auc_per_dose                   hour / liter                                          12   
auc_relative                   dimensionless                                         27   
aumc_inf                       gram * hour ** 2 / liter                               3   
clearance                      gram * milliliter / IU / hour                          7   
                               gram * milliliter / IU / hour / kilogram               6   
                               liter / hour                                         763   
                               liter / hour / kilogram                              818   
                               microgram * milliliter / hour / micromole              4   
                               milliliter / hour / micromole                          2   
                               milliliter / meter ** 2 / minute                       2   
clearance_intrinsic            liter / hour                                           2   
                               liter / hour / kilogram                                3   
clearance_intrinsic_unbound    liter / hour / kilogram                                2   
clearance_partial              liter / hour / kilogram                               51   
clearance_renal                liter / hour                                          83   
                               liter / hour / kilogram                               41   
clearance_renal_unbound        liter / hour / kilogram                                2   
...                                                                                 ...   
cmax                           mole / liter                                          14   
concentration                  gram / liter                                         137   
cumulative amount              gram                                                   7   
cumulative metabolic ratio     none                                                  48   
fraction_absorbed              dimensionless                                         12   
                               none                                                   6   
fraction_unbound               dimensionless                                          6   
kabs                           1 / minute                                            14   
kel                            1 / minute                                           703   
lag_absorption                 hour                                                   2   
metabolic ratio                dimensionless                                         26   
                               none                                   

In [12]:
this_data.data[this_data.data["measurement_type"]=="tmax"].groupby(["calculated"]).apply(len)

calculated
False    345
True     760
dtype: int64

### Merge
 - merge individuals and groups  --> all_subjects,
 - merge outputs and timecourses --> all_results, 
 - merge all_results with interventions and individuals  --> individuals_complete, 
 - merge all_results with interventions and groups  --> groups_complete, 
 - merge all_results with interventions and all_subjects -->  all_complete

In [13]:
prepocessed = Preprocessed()

In [14]:
prepocessed.read()
prepocessed.merge()
prepocessed.save()

/home/janekg89/Develop/Pycharm_Projects/pkdb_analysis/utils.py:427: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat([self.individuals.data,self.groups.data], keys=["individual","group"])
/home/janekg89/Develop/Pycharm_Projects/pkdb_analysis/utils.py:438: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat([self.outputs.data,self.timecourses.data], keys=["outputs","timecourses"])
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended